In [1]:
import numpy as np 
import pandas as pd 
import scipy as sc 
import sklearn as sk 
import matplotlib as plot

In [38]:
team_map = {"ATL" : "Atlanta Hawks", "BKN" : "Brooklyn Nets", "BOS": "Boston Celtics", "CHA" : "Charlotte Hornets", "CHI" : "Chicago Bulls", "CLE" : "Cleveland Cavaliers", "DAL" : "Dallas Mavericks", "DEN" : "Denver Nuggets", "DET" : "Detroit Pistons", "GSW" : "Golden State Warriors", "HOU" : "Houston Rockets", "IND" : "Indiana Pacers", "LAC" : "Los Angeles Clippers", "LAL" : "Los Angeles Lakers", "MEM" : "Memphis Grizzlies", "MIA" : "Miami Heat", "MIL" : "Milwaukee Bucks", "MIN" : "Minnesota Timberwolves","NOP" : "New Orleans Pelicans", "NYK" : "New York Knicks", "OKC" : "Oklahoma City Thunder", "ORL" : "Orlando Magic", "PHI" : "Philadelphia 76ers", "PHX" : "Phoenix Suns", "POR" : "Portland Trail Blazers", "SAC" : "Sacramento Kings", "SAS" : "San Antonio Spurs", "TOR" : "Toronto Raptors", "UTA" : "Utah Jazz", "WAS" : "Washington Wizards"}

df_team = pd.read_csv('../data/teams_stats/2019-20_nba_teams.csv')
df_player = pd.read_csv('../data/player_stats/2019-20_nba_players_new.csv')


In [39]:
df_team.head()

,Rk,Team,G,MP,FG,FGA,FG%,3P,3PA,3P%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1.0,Dallas Mavericks*,75,18175,3124,6772,0.461,1136,3095,0.367,...,0.779,791,2729,3520,1849,460,363,953,1462,8776
1,2.0,Milwaukee Bucks*,73,17595,3160,6638,0.476,1007,2840,0.355,...,0.742,691,3083,3774,1889,526,429,1102,1431,8663
2,3.0,Portland Trail Blazers*,74,17835,3122,6749,0.463,952,2525,0.377,...,0.804,754,2599,3353,1525,465,451,946,1606,8508
3,4.0,Houston Rockets*,72,17380,2936,6512,0.451,1126,3261,0.345,...,0.791,704,2484,3188,1557,627,371,1057,1566,8482
4,5.0,Los Angeles Clippers*,72,17380,2992,6425,0.466,895,2410,0.371,...,0.791,767,2664,3431,1708,511,338,1051,1594,8377


In [40]:
df_player.head()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Steven Adams/adamsst01,C,26,OKC,63,63,1680,283,478,...,0.582,207,376,583,146,51,67,94,122,684
1,2,Bam Adebayo/adebaba01,PF,22,MIA,72,72,2417,440,790,...,0.691,176,559,735,368,82,93,204,182,1146
2,3,LaMarcus Aldridge/aldrila01,C,34,SAS,53,53,1754,391,793,...,0.827,103,289,392,129,36,87,74,128,1001
3,4,Kyle Alexander/alexaky01,C,23,MIA,2,0,13,1,2,...,NaN,2,1,3,0,0,0,1,1,2
4,5,Nickeil Alexander-Walker/alexani01,SG,21,NOP,47,1,591,98,266,...,0.676,9,75,84,89,17,8,54,57,267


In [41]:
def stripAsterisk(s):
    if s[-1] == '*':
        return s[0:-1]
    else:
        return s

df_team['Team'] = df_team['Team'].apply(stripAsterisk)
df_team = df_team.dropna()
df_team.head()

,Rk,Team,G,MP,FG,FGA,FG%,3P,3PA,3P%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1.0,Dallas Mavericks,75,18175,3124,6772,0.461,1136,3095,0.367,...,0.779,791,2729,3520,1849,460,363,953,1462,8776
1,2.0,Milwaukee Bucks,73,17595,3160,6638,0.476,1007,2840,0.355,...,0.742,691,3083,3774,1889,526,429,1102,1431,8663
2,3.0,Portland Trail Blazers,74,17835,3122,6749,0.463,952,2525,0.377,...,0.804,754,2599,3353,1525,465,451,946,1606,8508
3,4.0,Houston Rockets,72,17380,2936,6512,0.451,1126,3261,0.345,...,0.791,704,2484,3188,1557,627,371,1057,1566,8482
4,5.0,Los Angeles Clippers,72,17380,2992,6425,0.466,895,2410,0.371,...,0.791,767,2664,3431,1708,511,338,1051,1594,8377


In [54]:
def nameOnly(s):
    new_s = s.split('/')
    return new_s[0]

df_player['Player'] = df_player['Player'].apply(nameOnly)
df_player = df_player.dropna()
df_player.head()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Steven Adams,C,26,OKC,63,63,1680,283,478,...,0.582,207,376,583,146,51,67,94,122,684
1,2,Bam Adebayo,PF,22,MIA,72,72,2417,440,790,...,0.691,176,559,735,368,82,93,204,182,1146
2,3,LaMarcus Aldridge,C,34,SAS,53,53,1754,391,793,...,0.827,103,289,392,129,36,87,74,128,1001
4,5,Nickeil Alexander-Walker,SG,21,NOP,47,1,591,98,266,...,0.676,9,75,84,89,17,8,54,57,267
5,6,Grayson Allen,SG,24,MEM,38,0,718,117,251,...,0.867,8,77,85,52,10,2,33,53,330
